In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
# Define the ticker and download data
ticker = 'SPY'
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

price_data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

print(price_data.head())
print(price_data.tail())

In [ ]:
# Trading Signal

# Calculate the % returns
price_data['daily_return'] = price_data['Close'].pct_change()

# Calculate return from previous day by shift
price_data['signal'] = price_data['daily_return'].shift(1)

# Clean data
price_data.dropna(inplace=True)

# Print the head and tail
print("First 5 rows")
print(price_data[['Close', 'daily_return', 'signal']].head())

print("Last 5 rows")
print(price_data[['Close', 'daily_return', 'signal']].tail())

In [ ]:
# BACKTEST WITH VECTOR AND STATISTICS
import vectorbt as vbt

long_threshold = 0.01
short_threshold = -0.01

#Entry
entries = price_data['signal'] > long_threshold
#Exit
exits = price_data['signal'] < short_threshold

portfolio = vbt.Portfolio.from_signals(
    close=price_data['Close'],
    open=price_data['Open'],
    entries = entries,
    exits = exits,
    init_cash = 100000,
    freq = 'D')

print(portfolio.stats())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Backtest Dashboard

print("Generating the complete backtest dashboard...")

# Data Preparation (Signals and Returns)
long_threshold = 0.01
short_threshold = -0.01
entries = price_data['signal'] > long_threshold
exits = price_data['signal'] < short_threshold

daily_returns = price_data['Close'].pct_change().dropna()
benchmark_cumulative_returns = (1 + daily_returns).cumprod() - 1
strategy_cumulative_returns = portfolio.cumulative_returns()
drawdown_series = portfolio.drawdown()['SPY']


#Three Subplots
fig, (ax1, ax2, ax3) = plt.subplots(
    nrows=3,
    ncols=1,
    figsize=(18, 10),
    sharex=True)
fig.suptitle('OFI Backtest Analysis', fontsize=20)

#Price with Signals (Top Chart)
ax1.set_title('SPY Price with Buy & Sell Signals', fontsize=14)
ax1.set_ylabel('Price ($)', fontsize=12)
ax1.plot(price_data.index, price_data['Close'], label='SPY Close Price', color='k')
ax1.plot(price_data.loc[entries].index, price_data.loc[entries]['Close'], linestyle='None', marker='^', markersize=4, color='green', label='Buy Signal')
ax1.plot(price_data.loc[exits].index, price_data.loc[exits]['Close'], linestyle='None', marker='v', markersize=4, color='red', label='Sell/Short Signal')
ax1.grid(True)
ax1.legend()

#Cumulative Returns (Middle Chart)
ax2.set_title('Strategy Performance vs. Buy & Hold Benchmark', fontsize=14)
ax2.set_ylabel('Cumulative Returns', fontsize=12)
ax2.plot(strategy_cumulative_returns.index, strategy_cumulative_returns, label='OFI Strategy', color='green')
ax2.plot(benchmark_cumulative_returns.index, benchmark_cumulative_returns, label='Benchmark (Buy & Hold SPY)', linestyle= '--')
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax2.grid(True)
ax2.legend()


#Drawdown Chart (Bottom Chart)
ax3.set_title('Portfolio Drawdown', fontsize=14)
ax3.set_ylabel('Drawdown', fontsize=12)
ax3.plot(drawdown_series.index, drawdown_series, color='red')
ax3.fill_between(drawdown_series.index, drawdown_series, 0, color='red', alpha=0.1)
ax3.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax3.grid(True)



plt.xlabel('Date', fontsize=12)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()